```
                            (#%(//***,.                                         
                          %@&&&&&&&&%&&&&&&#                                    
                       ,@@@@@&&%#(/*,,*(#%&&@&. ,#%&&&&&@&&#,                   
                      %&@@@@&&%(*,.....,*(#%&&@&&@@@@@&%&&@@@@@@@&              
                     /%@&@@@&%%(/******/(##%&&@@&@@&#((////(((#%&@@@@           
            /&@@@@@@&%@@@&@@&%%###(((#%%%&&&&&&&##%&#///*/(&@@@&%%&@@@#         
        (@@@@&/@@@@@@@@#%@@@@&&%%%%*/#&%#%&&&@@@@@@&#//&%                       
      @@@&&((##(%&@@@@@@&%@%@&@(%#%%%%%%,./(/.(/&((&@.                          
    &@@%//***,****(#%&@@@@@@@@%&*/#/.*(///*,//.#%%,&                            
   &@@(/**,,,,,,,,,,/&&@@@@@@@@&. ((/  /#%####%&&&(&                            
  &@@#(#%@@@&*        %@@@@@@@@&*#%&/%((((%&&%%&&@@&                            
 .@@@@@                   (#&&&&@@@@&&&//,,,(/(##/                             ,
 .@&                      %@@&&(%(,*(/,..*(#/(%#((#%(.                       ,.@
  .                   .(&%&@@@&%##**(#@/,  %@%%@#/#,%&&&                    ,.@@
                  *%&@@&(/#%&@&@@@#(.. .,,*/*&@@@///,%%&@%                 ,.@@@
                &@@@&&&%&@*,(#@@&@@@@@@(. ,&@&%@@&(,%,&&@@/              .,(@@@@
               &@@%%(/#%&@@,,*(%@%@@@@@@@&##/(%&&%&#/#&@@@&           ..*.@@@@@@
               @@@&%%%&&&@@@,%,&%&&&@@&%((&(%&@@@@%@#&&&&&@&.        ..*,@@@@@@@
               #@@@&%(((%&&@%#,(%%@@&@&%*@@@@@@@@@@@%@&&/(#&&*...  ...*,@@@@@@@@
               .%&&&(  ,(%&&@&*(&&&&&%/,&@&(@@&%&@@@&@@@&**#&&&,. ../*(@@@@@@@@&
         .......%&@@@&&&@@@@@&/&&&%%#/(&@&(%%@%%%%@&@&&@@&&./#%%* ./*&@@@@@@@@@@
##############(,*@@&&&&&&&@@@&&@@@@&#*#&&@&#%&%%#%%%@@&%@&&&#,(%&%*/@@@@@@@@@@@@
##############(*.@@@%%#&&&@@@&@&%#(*#&@&#(#&&%&&##(,%&&&%%&@&&&@%/&@@@@@@@@@@@@@
((##########(((/.%@@@#(#%&&@&&&%(*,(#%%&&(%#%#%&%#(,.#%&@@&@(,,%*#@@@@@@@@@@@@@@
//(##########(,...@@@@@&&%@@&&%#((#%&&&%&%%&%%%&&%    ##%&@@@@/*&%@@@@@@@@@@@@@@
(###########((/,,.(@@@&/,(@&&&&%%&&&&&%%&&&%%%%&&........%%%%**@@@@@@@@&@@@@@@@@
###(,,,,*/(((((*,(%@@@@@@@@&&&&&&&@&%/&&%#%%%%&&#./#(*//(((%*/@@@@@@@@@@@@@@@@@@
,*###(((((((((((#&&@@&@@@@&@@&&@@&&(&#**,**,,,********//((%*#@@@@@@@@@@@@@@@@@@@
///////////////(#%&&@@&@@@@@@@@@&(**,*,,,,,,,,,*%//%##(##(*&@@@@@@@@@@@@@@@@@@@@
///////////////////(##%&@@@@@@@&%******&/(##%###########/*@@@@@@@@@@@@@@@@@@@@@@
////////////////////((#%%%%&@@@&&#,(/#%*,*/*.&%########//@@@@@@@@@@@@@@@@@@@@@@&
%%(*/////////////////////(///////#@@@@@@@@/(%*,*/(.,&&*(@@@@@@@@@@@@@@@@@@@@@@(/
```
# Goblin Mode

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import cv2
import copy
import json
import math
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt

from time import time
from IPython.display import clear_output
from HelperFunctions import getFeatureVector, plot, getLocation
from dataloader.dataloader import DataLoader

In [39]:
# Initializing mediapipe pose class.
mp_pose = mp.solutions.pose

# Setting up the Pose function.
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

# Initializing mediapipe drawing class, useful for annotation.
mp_drawing = mp.solutions.drawing_utils 

In [3]:
myPath = os.getcwd()

myYogaPath = os.path.join(myPath, 'yoga')
myDataPath = os.path.join(myPath, 'data')

myYogaFolders = os.listdir(myYogaPath)
myDataFiles = os.listdir(myDataPath)

In [4]:
def center_chest(pose):
    CHEST_LOCATION_CARTESIAN = (getLocation(pose, 11) + getLocation(pose, 12)) / 2.0
    indexesToUse = [0, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
    formattedRow = np.empty(0)
    for index in indexesToUse:
        newTriple = getLocation(pose, index) - CHEST_LOCATION_CARTESIAN
        newTriple[1] *= -1
        formattedRow = np.append(formattedRow, newTriple)
    formattedRow.shape = 23, 3
    return formattedRow

def rotate_pose(pose):

    flip = 1
    if pose[1][2] < pose[2][2]:
        flip *= -1

    WAIST_CENTER = (pose[13] + pose[14]) / 2.0
    ogTheta = np.arctan2(WAIST_CENTER[0], WAIST_CENTER[1])

    # create the rotation matrix for x-axis
    theta_x = np.arctan2(pose[1][1], pose[1][0])
    R_x = np.array([[np.cos(theta_x), -np.sin(theta_x), 0],
                  [np.sin(theta_x), np.cos(theta_x), 0],
                  [0, 0, 1]])
    
    # rotate the point
    newPose = pose @ R_x

    # create the rotation matrix for z-axis
    theta_z = np.arctan2(newPose[1][0], newPose[1][2])
    R_z = np.array([[np.cos(theta_z), 0, np.sin(theta_z)],
                    [0, 1, 0],
                    [-np.sin(theta_z), 0, np.cos(theta_z)]])

    # rotate the point around z-axis
    newPose = newPose @ R_z

    WAIST_CENTER = (newPose[13] + newPose[14]) / 2.0
    newTheta = np.arctan2(WAIST_CENTER[0], WAIST_CENTER[1])

    # create the rotation matrix for x-axis
    theta_x = ogTheta*flip - newTheta
    R_x = np.array([[np.cos(theta_x), -np.sin(theta_x), 0],
                  [np.sin(theta_x), np.cos(theta_x), 0],
                  [0, 0, 1]])

    # rotate the point
    newPose = newPose @ R_x

    return newPose

def cart_plot(pose):

    fig = plt.figure(figsize = (6, 6))
    ax = plt.axes(projection='3d')

    x = [-0.5, -0.5, -0.5, -0.5, 0, 0.5, 0.5, 0.5, 0.5]
    y = [-0.5, -0.5, 0.5, 0.5, 0, -0.5, -0.5, 0.5, 0.5]
    z = [-0.5, 0.5, -0.5, 0.5, 0, -0.5, 0.5, -0.5, 0.5]

    ax.scatter3D(x, y, z, c = "black", alpha = 1)
    ax.scatter3D(pose[:,0], pose[:,1], pose[:,2], c = "white", s = 2.5, alpha = 1)
    ax.scatter3D(pose[:,0], pose[:,1], pose[:,2], c = "black", s = 4.5, alpha = 0.5)
    ax.scatter3D(pose[0,0], pose[0,1], pose[0,2], c = "black", s = 800, alpha = 0.5)
    ax.scatter3D(pose[0,0], pose[0,1], pose[0,2], c = "black", s = 600, alpha = 0.75)
    ax.scatter3D(pose[0,0], pose[0,1], pose[0,2], c = "black", s = 600, alpha = 0.75)

    halfConnections = [(0, 1), (0, 2), (1, 2), (13, 14)]
    for connection in halfConnections:
        rowA = pose[connection[0]]
        rowB = pose[connection[1]]
        mat = np.transpose([rowA, rowB])
        ax.plot3D(mat[0], mat[1], mat[2], c = "#444444", lw = 2, alpha = 1)

    leftHalfConnections = [(1, 3), (3, 5), (5, 7), (7, 9), (5, 9), (5, 11),
                           (1, 13), (13, 15), (15, 17), (17, 19), (19, 21), (17, 21)]
    
    for connection in leftHalfConnections:
        rowA = pose[connection[0]]
        rowB = pose[connection[1]]
        mat = np.transpose([rowA, rowB])
        ax.plot3D(mat[0], mat[1], mat[2], c = "#00CCCC", lw = 2.5, alpha = 1)
    
    rightHalfConnections = [(2, 4), (4, 6), (6, 8), (8, 10), (6, 10), (6, 12),
                            (2, 14), (14, 16), (16, 18), (18, 20), (20, 22), (18, 22)]
    for connection in rightHalfConnections:
        rowA = pose[connection[0]]
        rowB = pose[connection[1]]
        mat = np.transpose([rowA, rowB])
        ax.plot3D(mat[0], mat[1], mat[2], c = "#EEA000", lw = 2.5, alpha = 1)

    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")

    ax.view_init(90, -90)
    plt.show()


In [6]:
def yogaToVector():
    
    for folder in myYogaFolders:

        print(folder)
        allVectors = []

        posePath = os.path.join(myYogaPath, folder)
        myYogaImages = os.listdir(posePath)

        for imageName in myYogaImages:

            imagePath = os.path.join(posePath, imageName)
            
            imageRGB = None
            try:
                image = cv2.imread(imagePath)
                imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            except:
                continue

            results = pose.process(imageRGB)
            if results.pose_landmarks:
                featureVector = center_chest(results.pose_landmarks.landmark)
                allVectors.append(rotate_pose(featureVector))

        np.save(f'data/{folder}.npy', np.array(allVectors, dtype=float), allow_pickle=True)

def read_data():

    allVectors = {}
    for file in myDataFiles:
        posePath = os.path.join(myDataPath, file)
        allVectors[file[:-5]] = np.load(posePath, allow_pickle=True)
        allVectors[file[:-5]].shape = allVectors[file[:-5]].shape[0], allVectors[file[:-5]].shape[1] * allVectors[file[:-5]].shape[2]

    return allVectors


In [7]:
allVectors = read_data()

In [10]:
allVectors['Akarna_Dhanurasan'].shape[0]

66

In [11]:
allVectors['Akarna_Dhanurasan'].shape = allVectors['Akarna_Dhanurasan'].shape[0], allVectors['Akarna_Dhanurasan'].shape[1] * allVectors['Akarna_Dhanurasan'].shape[2]

In [ ]:
# scale of 0.4 is sometimes changed, some poses look really small ???
# make sure center between hips is directly below chest, move all point accordingly.

In [ ]:
for pose in allVectors:
    
    print(pose)

    avgPose = np.average(allVectors[pose], axis=0)
    stdPose = np.std(allVectors[pose], axis=0)

    cart_plot(avgPose)

    x = input()
    if x == '':
        break

In [ ]:
# Ignore this

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.cuda.get_device_name(0)

class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      self.conv2 = nn.Conv2d(32, 64, 3, 1)
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)
      self.fc1 = nn.Linear(9216, 128)
      self.fc2 = nn.Linear(128, 10)

    # x represents our data
    def forward(self, x):
      
      x = self.conv1(x)
      x = F.relu(x)

      x = self.conv2(x)
      x = F.relu(x)

      x = F.max_pool2d(x, 2)
      x = self.dropout1(x)

      x = torch.flatten(x, 1)
      
      x = self.fc1(x)
      x = F.relu(x)
      x = self.dropout2(x)
      x = self.fc2(x)

      # Apply softmax to x
      output = F.log_softmax(x, dim=1)
      return output
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:

pose = np.array([[0.23254013, 2.76807344, 5.73913384],
                 [0.08049627, 2.24089887, 1.57079633],
                 [0.08049627, 0.90069378, 4.71238898],
                 [0.18436866, 2.03073267, 1.63709208],
                 [0.16933819, 1.49463681, 4.69924022],
                 [0.34572825, 2.26337499, 1.6370426 ],
                 [0.32028896, 2.1859939 , 4.76248521],
                 [0.37376704, 2.29863436, 1.55370531],
                 [0.36639792, 2.22768258, 4.7704717 ],
                 [0.41289972, 2.38211349, 1.5451263 ],
                 [0.39590516, 2.29756873, 4.77119   ],
                 [0.37285039, 2.28861012, 1.58309893],
                 [0.35706384, 2.20045829, 4.77021712],
                 [0.31064981, 0.6889687 , 3.03864321],
                 [0.35416358, 0.52448185, 3.3566056 ],
                 [0.40594026, 1.05399902, 2.97361432],
                 [0.27849593, 0.87224705, 4.15043488],
                 [0.62015645, 0.97662971, 2.97564553],
                 [0.37988957, 1.07425651, 4.56016855],
                 [0.63418983, 0.98189344, 2.97366443],
                 [0.40404757, 1.1182807 , 4.56533306],
                 [0.5748927 , 1.23366494, 2.9687196 ],
                 [0.35414937, 1.3397443 , 4.72858835]])

theta = pose[1][1]
phi = pose[1][2]
R = pose[1][0]
pose[1] = np.array([R, 0, phi])

# Rotate the remaining rows accordingly
for i in range(2, len(pose)):
    R = pose[i][0]
    theta = pose[i][1]
    phi = pose[i][2]
    pose[i] = np.array([R, phi + np.pi if theta <= np.pi else phi - np.pi, np.pi - theta])
    
print(pose)

In [ ]:
# Corpse_Pose_or_Savasana_
# Bow_Pose_or_Dhanurasana_ <- why does it flip